## LAPPY_Modeleur_aerodynamique.ipynb*
Nothing to complete in this file.

This modeller provides functions to:

- assign input data, chosen by "aerodynamics" expert:
    -  **affecte_entrees_aerodynamique(avion)**
- calculate some aero characteristics:
    - **calcul_surfaces_mouillees(avion)**
    - **calcul_caracteristique_basse_vitesse(avion)**
    - **calcul_caracteristique_grande_vitesse(avion)**
    - **calcul_foyer_aero(avion)**
    - **calcul_aerodynamique(avion)** la succession des calculs précédents
- analyze that obtained lift-to-drag ratio is correct: 
    - **analyse_finesse(avion)** 



In [2]:
mach_data = [0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87]
konde_data = [17.5, 9.5, 6.3, 4, 2.6, 1.95, 1.4, 1, 0.8, 0.6, 0.55, 0.45, 0.4, 0.4, 0.4, 0.4]
nonde_data = [12.3, 11, 9.8, 8.7, 7.8, 7, 6.3, 5.6, 5.1, 4.6, 4.2, 3.7, 3.4, 3, 2.8, 2.5]

def affecte_entrees_aerodynamique(avion):
    #Présence bec interne voilure
    avion.nbeciv = 0
    #Présence bec externe voilure
    avion.nbecev = 1
    #Coefficient de frottement
    avion.kf = 0.004
    #Portance à trainée mini
    avion.czm = 0.1
    #Finesse seuil
    avion.finesse_seuil = 12
    return

def calcul_surfaces_mouillees(avion):
    #Surface mouillée voilure [m^2]
    avion.smv = 2*avion.sxv
    #Surface mouillée empennage [m^2]
    avion.smemp = 2*avion.sremp
    #Surface mouillée dérive [m^2]
    avion.smd = 2*avion.srd
    #Surface mouillée mats [m^2]
    avion.smmat = 2*avion.nbmo
    #Surface mouillée moteurs [m^2]
    avion.smmo = (pi*avion.diafanmo*avion.lmo)*avion.nbmo
    #Surface mouillée fuselage [m^2]
    avion.smfu = pi*avion.diafu/2*sqrt((avion.diafu/2)**2+avion.lpteavt**2)+(avion.lfu-avion.lbag-avion.lptearr-avion.lpteavt)*(2*pi*avion.diafu/2)+pi*(avion.diafu/2+avion.diafinfu/2)*sqrt(avion.lptearr**2+(avion.diafu/2-avion.diafinfu/2)**2)
    #Surface mouillée totale
    avion.smtot = avion.smv+avion.smemp+avion.smd+avion.smmat+avion.smmo+avion.smfu
    return
    
def calcul_caracteristique_basse_vitesse(avion):
    #Czmax voilure avec becs seuls
    if avion.nbeciv==1:
        avion.czmaxbv = 1.7
    else:
        avion.czmaxbv = 1.4
    #Czmax voilure avec becs et volets
    if avion.nbeciv==1:
        avion.czmaxbvv = 2.1
    else:
        avion.czmaxbvv = 1.9 
    avion.czmaxto = avion.czmaxbvv
    return
    
def calcul_caracteristique_grande_vitesse(avion):
    #Trainée de frottement
    avion.cx0 = avion.kf*avion.smtot/avion.srv
    #Ouverture polaire voilure pour flèche de 45 deg
    avion.kpol45 = 0.0046*avion.arv**2-0.0705*avion.arv+0.3823
    #Ouverture polaire voilure pour flèche de 30 deg
    avion.kpol30 = 0.0041*avion.arv**2-0.0654*avion.arv+0.3288
    #Ouverture polaire voilure
    avion.kpol = (45-avion.fbarv)/(45-30)*(avion.kpol30-avion.kpol45)+avion.kpol45
    #Nombre de Mach normal
    avion.machnorm = avion.mach_obj*sqrt(cos(avion.fbarv*pi/180))
    #Konde flèche nulle
    avion.konde = np.interp(avion.machnorm,mach_data,konde_data)
    #Nonde flèche nulle
    avion.nonde = np.interp(avion.machnorm,mach_data,nonde_data)
    #Trainée moyenne en mission
    avion.mis_cx = avion.cx0+avion.kpol*avion.mis_cz*(avion.mis_cz-2*avion.czm)+avion.konde*avion.mis_cz**(avion.nonde)*cos((avion.fbarv*pi/180)**2.5)
    #Finesse moyenne en mission
    avion.mis_fins = avion.mis_cz/(avion.cx0+avion.kpol*avion.mis_cz*(avion.mis_cz-2*avion.czm)+avion.konde*avion.mis_cz**avion.nonde*(cos(radians(avion.fbarv)**2.5)))
    return

def calcul_foyer_aero(avion):
     #X foyer voilure [m]
    avion.xfv = avion.xcmarv+0.25*avion.cmarv+avion.fbarv/10*1/100
    #Surface d'application foyer voilure [m^2]          
    avion.sfv = avion.srv
    #Czalpha voilure [rad-1]
    avion.czav = 2*pi*avion.arv/(2+sqrt(4+avion.arv*avion.arv/(cos(avion.f50rv*pi/180)*cos(avion.f50rv*pi/180))))
    #X foyer empennage [m]
    avion.xfemp = avion.xcmaremp+0.25*avion.cmaremp+avion.fbaemp/10*1/100
    #Surface d'application foyer empennage [m^2]           
    avion.sfemp = avion.sremp
    #Effet voilure sur empennage
    avion.epsemp = 0.5
    #Czalpha empennage [rad-1]
    avion.czaemp = 2*pi*avion.aremp/(2+sqrt(4+avion.aremp*avion.aremp/(cos(avion.f50emp*pi/180)*cos(avion.f50emp*pi/180))))*avion.epsemp
    #X foyer fuselage [m]
    avion.xffus = avion.xdiamaxfu
    #Surface d'application foyer fuselage [m^2]
    avion.sffus = pi*avion.diafu*avion.diafu/4
    #Czalpha fuselage [rad-1]
    avion.czafus = 2
    #X foyer nacelle [m]
    avion.xfnac = avion.xrnac
    #Surface d'application foyer nacelle [m^2]
    avion.sfnac = 2*avion.aenac
    #Czalpha nacelle [rad-1]
    avion.czanac = 2
    #X foyer interaction voilure fuselage [m]
    avion.xfivfus = avion.xrv+0.25*avion.crv+avion.fbarv/3.6*1/100
    #Surface d'application foyer interaction voilure fuselage [m^2]
    avion.sfivfus = avion.crv*avion.diafu
    #Czalpha interaction voilure fuselage [rad-1]
    avion.czaivfus = avion.czav
    #Position foyer avion [m]
    avion.xfoyeravion = (avion.xfv*avion.sfv*avion.czav+avion.xfemp*avion.sfemp*avion.czaemp+
                        avion.xffus*avion.sffus*avion.czafus+avion.xfnac*avion.sfnac*avion.czanac
                        +avion.xfivfus*avion.sfivfus*avion.czaivfus)/(avion.sfv*avion.czav+avion.sfemp*avion.czaemp+
                                                                   avion.sffus*avion.czafus+
                                                                   avion.sfnac*avion.czanac+
                                                                   avion.sfivfus*avion.czaivfus)
    # % CMA foyer avion
    avion.prcfoyeravion = (avion.xfoyeravion-avion.xcmarv)/avion.cmarv*100
    #X limite arrière [m]
    avion.xlimarr = avion.xfoyeravion-0.1*avion.cmarv
    return 

def calcul_aerodynamique(avion):
    calcul_surfaces_mouillees(avion)
    calcul_caracteristique_basse_vitesse(avion)
    calcul_caracteristique_grande_vitesse(avion)
    return

def analyse_finesse(avion):
    if avion.mis_fins < avion.finesse_seuil:
        avion.test_finesse=0
        #print("finesse trop faible :", avion.mis_fins)
    else:
        avion.test_finesse=1
        #print("finesse correcte :", avion.mis_fins)

